In [38]:
%matplotlib inline
import matplotlib
matplotlib.rc('font', **{'size': 12})

import pickle
import re
import datetime
from elasticsearch import Elasticsearch, helpers
from elasticsearch.helpers import scan
import datetime as dt

import numpy as np
import pandas as pd
import time
es = Elasticsearch([{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=60)

benchmark_indices = ['fts_*']
daysOfData=1

In [ ]:
#Get data from elasticsearch - all done transfers for 9 days
transfer_query = {
    "size": 0,
    "_source": ["src_rse", "dst_rse", "activity","bytes","submitted_at","started_at","transferred_at"],
    'query':{
            'bool':{
                   'should':[
                       { "term": {"_type": "transfer-done" } }
                   ]
            }
    }
                
}
data=[]
scroll = scan(client=es, index='fts_2017-01-31,fts_2017-02-01,fts_2017-02-02,fts_2017-02-03,fts_2017-02-04,fts_2017-02-05,fts_2017-02-06,fts_2017-02-07,fts_2017-02-08,', query=transfer_query, scroll='5m', timeout="5m", size=10000)
st = time.time()
for res in scroll:
    r = res['_source']
    r['submitted_at'] = (dt.datetime.strptime(r['submitted_at'].strip('Z').split('.')[0], '%Y-%m-%dT%H:%M:%S') - dt.datetime(1970,1,1)).total_seconds()
    r['started_at'] = (dt.datetime.strptime(r['started_at'].strip('Z').split('.')[0], '%Y-%m-%dT%H:%M:%S') - dt.datetime(1970,1,1)).total_seconds()
    r['transferred_at'] = (dt.datetime.strptime(r['transferred_at'].strip('Z').split('.')[0], '%Y-%m-%dT%H:%M:%S') - dt.datetime(1970,1,1)).total_seconds()
    data.append([r['src_rse'],r['dst_rse'],r['activity'],r['bytes'],r['submitted_at'],r['started_at'],r['transferred_at']])
print(time.time() - st)
data = pd.DataFrame(data, columns=['SRC', 'DST', 'ACT', 'SIZE', 'SUBMITTED', 'STARTED', 'ENDED'])
data['QUEUE_T'] = data.STARTED - data.SUBMITTED
data['NET_T'] = data.ENDED - data.STARTED
print(len(data))
data.to_csv('transfers_2017-02-01_07.csv',index=False)

3497.832622051239
27734735


In [4]:
import matplotlib.pyplot as plt

#plt.plot(data.STARTED.values[:1500000], (data.ENDED - data.STARTED).values[:1500000], '.')
#fig = plt.gcf()
#fig.set_size_inches(17, 8)

In [24]:
#Create the model - Return the average queue time and network time for each link
# filter outliers (queue_t > 170 hours and net_t > 200 minutes)
data = data.where(data.QUEUE_T/60/60 < 170).dropna()
data = data.where(data.NET_T/60 < 200).dropna()
links = {}
for index,row in data.iterrows():
    link = row.SRC + ' ' + row.DST + ' ' + row.ACT
    if link not in links.keys():
        links[link] = 0
        cut = data.where(data.SRC == row.SRC).dropna()
        cut = cut.where(cut.DST == row.DST).dropna()
        cut = cut.where(cut.ACT == row.ACT).dropna()
        links[link] = (cut.QUEUE_T.mean(), cut.NET_T.mean(), len(cut))
        #print (link,cut.QUEUE_T.mean(), cut.NET_T.mean(), len(cut), sep='  ')

In [40]:
d = data.where(data.STARTED >= 1485907200).dropna()
d = d.where(d.STARTED <= 1486512000).dropna()
len(d)

20867050

In [29]:
len(links)

9492

In [25]:
import pickle
pickle.dump(links, open('dumb_model2.pickle','wb'))